


Agentic RAG with ReAct Pattern

This exercise demonstrates an advanced RAG system using the ReAct (Reasoning + Acting) pattern.
The system can use multiple tools to gather information:
- Vector database search with reranking
- Web search for current information
- Company director information extraction with LinkedIn lookups

The objective is to answer complex multi-part questions by intelligently using these tools.

# How to Run this Notebook?

1. Generate API key for OpenAI (ChatGPT): https://platform.openai.com/settings/organization/api-keys
   Make sure to save the API key.

2. Generate API key for SerpAPI (for web search): https://serpapi.com/users/sign_up?plan=free

3. Generate API key for LangChain to access LangSmith (for tracing): https://smith.langchain.com/settings

4. Click on the key icon in the left menu bar of this Notebook

5. Click + Add new secret
   - Add OpenAI key:
     * Under Name: OPEN_AI_KEY
     * Under Value: Your OpenAI key
   - Add SerpAPI key:
     * Under Name: SERP_KEY
     * Under Value: Your SerpAPI key
- Add LangChain key:
     * Under Name: LANG_KEY
     * Under Value: Your LangSmith key

6. Enable access to the keys for this notebook by toggling the radio buttons.
7. Close the Secrets section once done.
8. Click Run all under the Runtime menu to execute this notebook.

# Basic Setup

## Install Frameworks

In [ ]:
!pip install langchain langchain_core langchain_community langchain_openai langchain_classic langchain_text_splitters faiss-cpu openai flashrank google-search-results python-dotenv -U

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
import langchain
import langchain_core
import langchain_community
import openai

print(f"langchain version: {langchain.__version__}")
print(f"langchain_core version: {langchain_core.__version__}")
print(f"langchain_community version: {langchain_community.__version__}")
print(f"openai version: {openai.__version__}")

langchain version: 1.0.5
langchain_core version: 1.0.5
langchain_community version: 0.4.1
openai version: 2.8.0


## API Keys Setup

Set up API keys for OpenAI, SerpAPI, and LangChain.

Important: You need to setup your API keys in Google Colab's Secrets manager or in a .env file for local execution.

In [ ]:
import os

# Check if we're in a Colab environment
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False


if IN_COLAB:
    from google.colab import userdata
    # Set environment variables from Colab secrets
    os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')
    os.environ["SERPAPI_API_KEY"] = userdata.get('SERP_KEY')
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANG_KEY')
else:
    # For local execution, load from .env file
    from dotenv import load_dotenv
    # Try to load .env from current directory first, then fall back to specified path
    load_dotenv()  # Loads from current directory

    # Set environment variables with validation
    openai_key = os.getenv('OPENAI_KEY')
    serpapi_key = os.getenv('SERPAPI_KEY')
    langchain_key = os.getenv('LANGCHAIN_KEY')


## LangChain Tracing Configuration

Configure LangSmith for tracing and debugging the agent's behavior.
This helps us see how the agent reasons and which tools it uses.

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "4c_agentic_rag_complex_fall_2025"

## Import Libraries

Import all necessary libraries for the agentic RAG pipeline.

In [ ]:
# Standard library imports
import re
from typing import List, Dict, Any, Tuple
from pprint import pprint

# LangChain core imports
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_core.tools import BaseTool

# LangChain community imports
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangChain OpenAI imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Flashrank for reranking
from flashrank import Ranker, RerankRequest
import langchain_community.document_compressors.flashrank_rerank as fr_mod
fr_mod.RerankRequest = RerankRequest
from langchain_community.document_compressors import FlashrankRerank

# For retriever with compression
from langchain_classic.retrievers import ContextualCompressionRetriever

# Agent imports
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# Configuration Dictionary

Contains all settings for the agentic RAG system, including:
- Document processing parameters
- Embedding model settings
- Vector store and retrieval settings
- Reranker configuration
- ReAct agent settings
- Tool configurations
- Prompt templates

In [ ]:
defaultConfig = {
    # Document processing settings
    "chunkSize": 500,
    "chunkOverlap": 50,
    "userAgentHeader": "YourCompany-ResearchBot/1.0 (your@email.com)",

    # Embedding model (OpenAI)
    "embeddingModelName": "text-embedding-3-small",

    # Vector store settings
    "numRetrievedDocuments": 12,

    # Document formatter settings
    "numSelectedDocuments": 12,

    # Reranker settings (Flashrank)
    "rerankerModel": "ms-marco-TinyBERT-L-2-v2",
    "numRerankedDocuments": 5,

    # Model settings for answer generation
    "ragAnswerModel": "gpt-4o",
    "ragAnswerModelTemperature": 0.7,

    # URLs to process - Multiple 10-K filings
    "companyFilingUrls": [
        ("Tesla", "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm"),
        ("General Motors", "https://www.sec.gov/Archives/edgar/data/1467858/000146785824000031/gm-20231231.htm")
    ],

    # RAG prompt template
    "ragPromptTemplate": """
    Give an answer for the Question using only the given Context. Use information relevant to the query from the entire context.
    Provide a detailed answer with thorough explanations, avoiding summaries.

    Question: {question}

    Context: {context}

    Answer:
    """,

    # ReAct Agent settings
    "reactModelName": "gpt-4o",
    "reactModelTemperature": 0,

    "reactPromptTemplate": """Your task is to gather relevant information to build context for the question. Focus on collecting details related to the question.
    Gather as much context as possible before formulating your answer.

    You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer

    Thought: you should always think about what to do

    Action: the action to take, should be one of [{tool_names}]

    Action Input: the input to the action

    Observation: the result of the action

    ... (this Thought/Action/Action Input/Observation can repeat N times)

    Thought: I now know the final answer

    Final Answer: the final answer to the question.

    Follow these steps:

    Begin!

    Question: {input}

    Thought:{agent_scratchpad}
    """,

    "reactVerbosity": True,

    # Name Extraction settings
    "nameExtractionModel": "gpt-4o-mini",
    "nameExtractionModelTemperature": 0.4,
    "nameExtractionPrompt": """
    Extract and list the names of all individuals with the title 'Director' from the following text, excluding any additional information such as dates or signatures.
    Present the names as a simple, comma-separated list.

    {text}
    """,

    # Tool settings
    "useDirectorTool": True,
    "directorToolName": "Company Directors Information",
    "directorToolDescription": "Retrieve the names of company directors for a chosen company. Optionally, their LinkedIn handles can also be included. Use the format: company_name, true/false.",

    "useWebTool": True,
    "webToolName": "WebSearch",
    "webToolDescription": "Performs a web search on the query.",
    "numWebToolResults": 3,

    "useRetrieverTool": True,
    "retrieverToolName": "Vector Reranker Search",
    "retrieverToolDescription": "Retrieves information from an embedding based vector DB containing financial data and company information. Structure query as a sentence.",
    "numRetrieverToolResults": 3
}

In [ ]:
config = defaultConfig.copy()

# Document Processing Functions

Functions to load and process company 10-K filings from SEC website.

In [ ]:
def load_and_process_filings(urls: List[Tuple[str, str]], config: Dict[str, Any]) -> Tuple[List[Document], Dict[str, str]]:
    """
    Load and process company filings from URLs.

    Args:
        urls: List of tuples (company_name, url)
        config: Configuration dictionary

    Returns:
        Tuple of (processed_chunks, director_sections)
    """
    processed_chunks = []
    director_sections = {}

    # Create text splitter
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunkSize"],
        chunk_overlap=config["chunkOverlap"]
    )

    for company, url in urls:
        try:
            print(f"Loading {company} filing from {url}")

            # Load document using WebBaseLoader
            loader = WebBaseLoader(
                url,
                header_template={'User-Agent': config["userAgentHeader"]}
            )
            docs = loader.load()

            # Store last 1000 characters for director extraction
            if docs and len(docs) > 0:
                director_sections[company] = docs[0].page_content[-1000:]

            # Split documents into chunks
            chunks = splitter.transform_documents(docs)

            # Add company metadata to each chunk
            for chunk in chunks:
                chunk.metadata["company"] = company

            processed_chunks.extend(chunks)
            print(f"Processed {len(chunks)} chunks from {company}")

        except Exception as e:
            print(f"Error processing {company} from {url}: {str(e)}")
            continue

    print(f"Total processed chunks: {len(processed_chunks)}")
    return processed_chunks, director_sections

# Vector Store and Retriever Functions

Functions to create the vector store and retriever with reranking capability.

In [ ]:
def create_vector_store(chunks: List[Document], config: Dict[str, Any]) -> FAISS:
    """
    Create a FAISS vector store with OpenAI embeddings.

    Args:
        chunks: List of document chunks
        config: Configuration dictionary

    Returns:
        FAISS vector store
    """
    print("Creating vector store with OpenAI embeddings...")

    # Create embedding function
    embedding_function = OpenAIEmbeddings(
        model=config["embeddingModelName"]
    )

    # Create FAISS vector store
    vectorstore = FAISS.from_documents(chunks, embedding_function)

    print(f"Vector store created successfully")
    print(f"Number of vectors: {vectorstore.index.ntotal}")
    print(f"Vector dimension: {vectorstore.index.d}")

    return vectorstore

In [ ]:
def create_retriever_with_reranking(vectorstore: FAISS, config: Dict[str, Any]):
    """
    Create a retriever with Flashrank reranking.

    Args:
        vectorstore: FAISS vector store
        config: Configuration dictionary

    Returns:
        ContextualCompressionRetriever with reranking
    """
    print("Creating retriever with Flashrank reranking...")

    # Create base retriever
    base_retriever = vectorstore.as_retriever(
        search_kwargs={"k": config["numRetrievedDocuments"]}
    )

    # Create Flashrank reranker
    model_name = config["rerankerModel"]
    top_n = config["numRerankedDocuments"]

    ranker_client = Ranker(model_name=model_name)
    reranker = FlashrankRerank(client=ranker_client, model=model_name, top_n=top_n)

    # Create compression retriever with reranker
    compression_retriever = ContextualCompressionRetriever(
        base_retriever=base_retriever,
        base_compressor=reranker
    )

    print("Retriever with reranking created successfully")
    return compression_retriever

# Custom Tool Classes

Three custom tools for the ReAct agent:
1. CompanyDirectorsTool - Extracts director names and finds LinkedIn profiles
2. WebSearchTool - Performs web searches using SerpAPI
3. VectorRerankerSearchTool - Searches the vector database with reranking

In [ ]:
# Cache for LinkedIn lookups to avoid repeated API calls
linkedin_cache = {}



Tool to retrieve company director names and LinkedIn profiles.

In [ ]:
class CompanyDirectorsTool(BaseTool):
    """Tool to retrieve company director names and LinkedIn profiles."""

    name: str = "Company Directors Information"
    description: str = "Retrieve the names of company directors for a chosen company. Optionally, their LinkedIn handles can also be included. Use the format: company_name, true/false."

    # Custom attributes
    director_sections: Dict[str, str] = {}
    config: Dict[str, Any] = {}

    def __init__(self, director_sections: Dict[str, str], config: Dict[str, Any]):
        """Initialize the tool with director sections and config."""
        # Update description with available companies
        available_companies = list(director_sections.keys())
        updated_description = f"{config['directorToolDescription']} Available companies: {', '.join(available_companies)}"

        super().__init__(
            director_sections=director_sections,
            config=config,
            description=updated_description
        )

    def _run(self, query: str) -> str:
        """Execute the tool to get director information."""
        try:
            # Parse input
            parts = query.split(',')
            company_name = parts[0].strip()
            include_linkedin = parts[1].strip().lower() == 'true' if len(parts) > 1 else True

            # Get director section
            company_snippet = self.director_sections.get(company_name)
            if not company_snippet:
                return f"No director information found for {company_name}"

            # Extract director names using LLM
            director_names = self._extract_director_names(company_snippet)

            if not director_names:
                return f"Could not extract director names for {company_name}"

            # Get LinkedIn handles if requested
            if include_linkedin:
                director_handles = []
                for name in director_names:
                    linkedin_handle = self._get_linkedin_handle(name, company_name)
                    director_handles.append(f"{name} (LinkedIn: {linkedin_handle})")

                return f"Directors of {company_name}: {'; '.join(director_handles)}"
            else:
                return f"Directors of {company_name}: {', '.join(director_names)}"

        except Exception as e:
            return f"Error retrieving director information: {str(e)}"

    def _extract_director_names(self, text: str) -> List[str]:
        """Extract director names from text using LLM."""
        try:
            llm = ChatOpenAI(
                model=self.config["nameExtractionModel"],
                temperature=self.config["nameExtractionModelTemperature"]
            )
            parser = CommaSeparatedListOutputParser()
            prompt = PromptTemplate.from_template(self.config["nameExtractionPrompt"])

            chain = prompt | llm | parser
            names = chain.invoke({"text": text})
            return names

        except Exception as e:
            print(f"Error extracting names: {str(e)}")
            return []

    def _get_linkedin_handle(self, name: str, company: str) -> str:
        """Get LinkedIn handle for a director."""
        cache_key = f"{name}_{company}"

        # Check cache first
        if cache_key in linkedin_cache:
            return linkedin_cache[cache_key]

        try:
            # Use SerpAPI to search LinkedIn
            search = SerpAPIWrapper()
            results = search.results(f'"{name}" {company} site:linkedin.com/in/')

            # Extract link from results
            link = results.get("organic_results", [{}])[0].get("link", "Profile not found")

            # Cache the result
            linkedin_cache[cache_key] = link
            return link

        except Exception as e:
            return f"Error finding LinkedIn profile: {str(e)}"

In [ ]:
class WebSearchTool(BaseTool):
    """Tool to perform web searches using SerpAPI."""

    name: str = "WebSearch"
    description: str = "Performs a web search on the query."

    # Custom attributes
    config: Dict[str, Any] = {}

    def __init__(self, config: Dict[str, Any]):
        """Initialize the tool with configuration."""
        super().__init__(config=config)

    def _run(self, query: str) -> str:
        """Execute the web search."""
        try:
            search = SerpAPIWrapper()
            results = search.results(query)
            return self._format_results(results)

        except Exception as e:
            return f"Error performing web search: {str(e)}"

    def _format_results(self, results: Dict) -> str:
        """Format search results into readable text."""
        formatted = []
        num_results = self.config["numWebToolResults"]

        for result in results.get("organic_results", [])[:num_results]:
            formatted.append(
                f"Title: {result.get('title', 'N/A')}\n"
                f"Snippet: {result.get('snippet', 'N/A')}\n"
                f"Link: {result.get('link', 'N/A')}"
            )

        return "\n\n".join(formatted)

In [ ]:
class VectorRerankerSearchTool(BaseTool):
    """Tool to search the vector database with reranking."""

    name: str = "Vector Reranker Search"
    description: str = "Retrieves information from an embedding based vector DB containing financial data and company information. Structure query as a sentence."

    # Custom attributes
    retriever: Any = None
    config: Dict[str, Any] = {}

    def __init__(self, retriever: Any, config: Dict[str, Any]):
        """Initialize the tool with retriever and config."""
        super().__init__(retriever=retriever, config=config)

    def _run(self, query: str) -> str:
        """Execute the vector search with reranking."""
        try:
            # Retrieve documents
            docs = self.retriever.invoke(query)

            # Format and return top results
            num_results = self.config["numRetrieverToolResults"]
            formatted_docs = []

            for doc in docs[:num_results]:
                company = doc.metadata.get('company', '')
                content = doc.page_content
                formatted_docs.append(f"{company}\n{content}")

            return "\n\n".join(formatted_docs)

        except Exception as e:
            return f"Error retrieving documents: {str(e)}"

# Agent Setup Functions

Functions to create and configure the ReAct agent with all tools.

In [ ]:
def create_tools(config: Dict[str, Any], director_sections: Dict[str, str], retriever) -> List[BaseTool]:
    """
    Create all tools for the ReAct agent.

    Args:
        config: Configuration dictionary
        director_sections: Dictionary mapping company names to director text sections
        retriever: Retriever with reranking

    Returns:
        List of tools
    """
    tools = []

    # Add Company Directors Tool
    if config.get("useDirectorTool", False):
        director_tool = CompanyDirectorsTool(director_sections, config)
        tools.append(director_tool)
        print(f"Added tool: {director_tool.name}")

    # Add Web Search Tool
    if config.get("useWebTool", False):
        web_tool = WebSearchTool(config)
        tools.append(web_tool)
        print(f"Added tool: {web_tool.name}")

    # Add Vector Reranker Search Tool
    if config.get("useRetrieverTool", False):
        retriever_tool = VectorRerankerSearchTool(retriever, config)
        tools.append(retriever_tool)
        print(f"Added tool: {retriever_tool.name}")

    print(f"Total tools created: {len(tools)}")
    return tools

In [ ]:
def create_react_agent_executor(tools: List[BaseTool], config: Dict[str, Any]) -> AgentExecutor:
    """
    Create a ReAct agent executor with the specified tools.

    Args:
        tools: List of tools for the agent
        config: Configuration dictionary

    Returns:
        AgentExecutor
    """
    print("Creating ReAct agent...")

    # Create LLM for the agent
    llm = ChatOpenAI(
        model=config["reactModelName"],
        temperature=config["reactModelTemperature"]
    )

    # Create prompt template
    prompt = PromptTemplate.from_template(config["reactPromptTemplate"])

    # Create ReAct agent
    agent = create_react_agent(llm, tools, prompt)

    # Create agent executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=config.get("reactVerbosity", True),
        handle_parsing_errors=True
    )

    print("ReAct agent created successfully")
    return agent_executor

# Main Execution Flow

Load documents, create vector store, set up tools, and run the agent.

In [ ]:
# Step 1: Load and process documents
print("=" * 60)
print("STEP 1: Loading and Processing Documents")
print("=" * 60)
processed_chunks, director_sections = load_and_process_filings(
    config["companyFilingUrls"],
    config
)

# Step 2: Create vector store
print("\n" + "=" * 60)
print("STEP 2: Creating Vector Store")
print("=" * 60)
vectorstore = create_vector_store(processed_chunks, config)

# Step 3: Create retriever with reranking
print("\n" + "=" * 60)
print("STEP 3: Creating Retriever with Reranking")
print("=" * 60)
retriever = create_retriever_with_reranking(vectorstore, config)

# Step 4: Create tools
print("\n" + "=" * 60)
print("STEP 4: Creating Tools for Agent")
print("=" * 60)
tools = create_tools(config, director_sections, retriever)

# Step 5: Create ReAct agent
print("\n" + "=" * 60)
print("STEP 5: Creating ReAct Agent")
print("=" * 60)
agent_executor = create_react_agent_executor(tools, config)

# Step 6: Ask a complex question
print("\n" + "=" * 60)
print("STEP 6: Running Agent with Complex Query")
print("=" * 60)

question = "Who are the directors of Tesla. What are their linkedin handles? What are the financial goals of tesla this year. What is the next auto show that Tesla will participate in."

print(f"\nQuestion: {question}\n")
print("Agent is working...\n")

response = agent_executor.invoke({"input": question})

print("\n" + "=" * 60)
print("FINAL ANSWER")
print("=" * 60)
print(response['output'])

STEP 1: Loading and Processing Documents
Loading Tesla filing from https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm
Processed 942 chunks from Tesla
Loading General Motors filing from https://www.sec.gov/Archives/edgar/data/1467858/000146785824000031/gm-20231231.htm
Processed 1085 chunks from General Motors
Total processed chunks: 2027

STEP 2: Creating Vector Store
Creating vector store with OpenAI embeddings...
Vector store created successfully
Number of vectors: 2027
Vector dimension: 1536

STEP 3: Creating Retriever with Reranking
Creating retriever with Flashrank reranking...


ms-marco-TinyBERT-L-2-v2.zip: 100%|██████████| 3.26M/3.26M [00:00<00:00, 13.5MiB/s]


Retriever with reranking created successfully

STEP 4: Creating Tools for Agent
Added tool: Company Directors Information
Added tool: WebSearch
Added tool: Vector Reranker Search
Total tools created: 3

STEP 5: Creating ReAct Agent
Creating ReAct agent...
ReAct agent created successfully

STEP 6: Running Agent with Complex Query

Question: Who are the directors of Tesla. What are their linkedin handles? What are the financial goals of tesla this year. What is the next auto show that Tesla will participate in.

Agent is working...



> Entering new AgentExecutor chain...
To answer the question, I need to gather information on the directors of Tesla, their LinkedIn handles, Tesla's financial goals for this year, and the next auto show Tesla will participate in.

Action: Company Directors Information
Action Input: Tesla, true
Directors of Tesla: Elon Musk (LinkedIn: https://www.linkedin.com/in/elon-musk-tesla-company-b7b00879); Robyn Denholm (LinkedIn: https://au.linkedin.com/in/robyn-den